In [ ]:
!pip install tensorflow-hub
!pip install tensorflow-datasets

In [ ]:

!pip install --upgrade tensorflow tensorflow-hub


import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 3.0 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.0
    Uninstalling tensorflow-2.17.0:
      Successfully uninstalled tensorflow-2.17.0


In [ ]:
print(f"wersja: {tf.__version__}")
print(f"wersja hub: {hub.__version__}")
print(f"wersja tfds: {tfds.__version__}")
print(f"Eager execution: {tf.executing_eagerly()}")
print(f"GPU available: {tf.test.is_gpu_available()}")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


wersja: 2.17.1
wersja hub: 0.16.1
wersja tfds: 4.9.6
Eager execution: True
GPU available: False


In [ ]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.72JOBE_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.72JOBE_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.72JOBE_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [ ]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [ ]:
model = tf.keras.Sequential()
# Ensure compatibility by wrapping the hub_layer in a Lambda layer if necessary
model.add(tf.keras.layers.Lambda(lambda x: hub_layer(x))) # Wrap hub_layer for compatibility
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 69ms/step - accuracy: 0.4984 - loss: 0.6862 - val_accuracy: 0.5029 - val_loss: 0.6580
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.5231 - loss: 0.6494 - val_accuracy: 0.5609 - val_loss: 0.6283
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - accuracy: 0.5807 - loss: 0.6196 - val_accuracy: 0.6156 - val_loss: 0.6036
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 89ms/step - accuracy: 0.6231 - loss: 0.6000 - val_accuracy: 0.6407 - val_loss: 0.5845
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.6571 - loss: 0.5807 - val_accuracy: 0.6608 - val_loss: 0.5692
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - accuracy: 0.6707 - loss: 0.5646 - val_accuracy: 0.6825 - val_loss: 0.5570
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - accuracy: 0.6853 - loss: 0.5571 - val_accuracy: 0.6945 - val_loss: 0.5474
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - accuracy: 0.6993 - loss: 0.5451 - val_accuracy: 0.7069 - v

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 2s - 48ms/step - accuracy: 0.7152 - loss: 0.5390
loss: 0.539
compile_metrics: 0.715


In [19]:
import numpy as np

# Assuming 'hub_layer' is a function that converts text to embeddings
# (e.g., from TensorFlow Hub)
reviews = [
    "This movie was absolutely amazing! The acting was superb and the plot kept me on the edge of my seat.",
    "A complete waste of time. The storyline was confusing and the acting was terrible.",
    "I enjoyed this film, but it wasn't perfect. Some parts were a bit slow, but overall it was entertaining.",
    "The cinematography was breathtaking, but the story lacked depth. A visually stunning film with a weak narrative.",
    "A truly captivating movie that will stay with you long after the credits roll. Highly recommend!"
]

revnp = np.array(reviews)
revnp = tf.convert_to_tensor(revnp)


# Pass the raw reviews directly to model.predict
predictions = model.predict(revnp)


for i, review in enumerate(revnp):
    print(f"Review: {review}")
    print(f"Prediction: {predictions[i][0]}")  # Access the prediction value
    print("----")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
Review: b'This movie was absolutely amazing! The acting was superb and the plot kept me on the edge of my seat.'
Prediction: 0.5346536040306091
----
Review: b'A complete waste of time. The storyline was confusing and the acting was terrible.'
Prediction: -0.29809534549713135
----
Review: b"I enjoyed this film, but it wasn't perfect. Some parts were a bit slow, but overall it was entertaining."
Prediction: 0.3609486520290375
----
Review: b'The cinematography was breathtaking, but the story lacked depth. A visually stunning film with a weak narrative.'
Prediction: 0.44515594840049744
----
Review: b'A truly captivating movie that will stay with you long after the credits roll. Highly recommend!'
Prediction: 1.2577040195465088
----
